# STAC client

## STAC API client
EODAG can perform an item search over a STAC compliant API. Found STAC items are returned as [EOProduct](../../api_reference/eoproduct.rst#eodag.api.product._product.EOProduct) objects with STAC metadata mapped to OGC OpenSearch Extension for Earth Observation.

EODAG comes with already configured providers, but you can also add new ones dynamically.

In [1]:
import os
from eodag import EODataAccessGateway, setup_logging
# To have some basic feedback on what eodag is doing, we configure logging to output minimum information
setup_logging(verbose=2)

dag = EODataAccessGateway()

2025-10-19 16:47:01,574 eodag.config                     [INFO    ] Loading user configuration from: /home/sylvain/.config/eodag/eodag.yml
2025-10-19 16:47:01,814 eodag.core                       [INFO    ] Locations configuration loaded from /home/sylvain/.config/eodag/locations.yml


List already configured providers providing a STAC API compliant service:

In [2]:
[p.name for p in dag.providers_config.values() if hasattr(p, "search") and p.search.type == 'StacSearch']

['usgs_satapi_aws',
 'earth_search',
 'earth_search_gcs',
 'planetary_computer',
 'geodes',
 'fedeo_ceda']

Then, let's update EODAG's configuration with a new STAC provider (West University of Timisoara STAC catalog) using [add_provider()](../../api_reference/core.rst#eodag.api.core.EODataAccessGateway.add_provider) method:

In [3]:
dag.add_provider("uvt", "https://stac.sage.uvt.ro/search")

2025-10-19 16:49:12,431 eodag.config                     [INFO    ] uvt: unknown provider found in user conf, trying to use provided configuration


List available collections for this provider (using [collections discovery](../api_user_guide/1_providers_products_available.ipynb#Collections-discovery) in background):

In [4]:
[pt["ID"] for pt in dag.list_collections("uvt")]

2025-10-19 16:49:14,241 eodag.config                     [INFO    ] Fetching external collections from https://cs-si.github.io/eodag/eodag/resources/ext_collections.json
2025-10-19 16:49:15,404 eodag.search.qssearch            [INFO    ] Fetching collections: https://stac.sage.uvt.ro/search/../collections


['ancpi-judete-mnt',
 'clc-2018-20',
 'clc-2018-20-vector',
 'copernicus-dem-30',
 'esa-worldcover-2020',
 'esa-worldcover-2021',
 'maja-cams',
 'sentinel-1-grd',
 'sentinel-1-rtc',
 'sentinel-2-l1c',
 'sentinel-2-l1c-ndvi',
 'sentinel-2-l2a-maja']

Search some *sentinel-2-l1c* products over Romania:

In [5]:
prods_S2L1C = dag.search(collection="sentinel-2-l1c", locations=dict(country="ROU"), start="2020-05-01", end="2020-05-15", items_per_page=10, count=True)
prods_S2L1C

2025-10-19 16:49:26,022 eodag.core                       [INFO    ] Searching on provider uvt
2025-10-19 16:49:26,025 eodag.search.qssearch            [INFO    ] Fetching queryables: https://stac.sage.uvt.ro/search/../collections/sentinel-2-l1c/queryables
2025-10-19 16:49:26,529 eodag.search.qssearch            [INFO    ] Sending search request: https://stac.sage.uvt.ro/search


SearchResult([EOProduct(id=S2B_MSIL1C_20200514T094029_N0209_R036_T34TET_20200514T114435, provider=uvt),
              EOProduct(id=S2B_MSIL1C_20200514T094029_N0209_R036_T34TES_20200514T114435, provider=uvt),
              EOProduct(id=S2B_MSIL1C_20200514T094029_N0209_R036_T34TER_20200514T114435, provider=uvt),
              EOProduct(id=S2B_MSIL1C_20200514T094029_N0209_R036_T34TDS_20200514T114435, provider=uvt),
              EOProduct(id=S2B_MSIL1C_20200514T094029_N0209_R036_T34TDR_20200514T114435, provider=uvt),
              EOProduct(id=S2A_MSIL1C_20200513T092031_N0209_R093_T35UNP_20200513T112445, provider=uvt),
              EOProduct(id=S2A_MSIL1C_20200513T092031_N0209_R093_T35UMP_20200513T112445, provider=uvt),
              EOProduct(id=S2A_MSIL1C_20200513T092031_N0209_R093_T35ULP_20200513T112445, provider=uvt),
              EOProduct(id=S2A_MSIL1C_20200513T092031_N0209_R093_T35TNN_20200513T112445, provider=uvt),
              EOProduct(id=S2A_MSIL1C_20200513T092031_N0209_R093_T35TMN_20200513T112445, provider=uvt)])

Filter over any item property using crunchers:

In [6]:
from eodag.plugins.crunch.filter_property import FilterProperty

prods_S2L1C_filtered = prods_S2L1C.crunch(FilterProperty({"eo:cloud_cover": 10, "operator": "lt"}))
prods_S2L1C_filtered

2025-10-19 16:49:30,343 eodag.crunch.property            [INFO    ] Finished filtering products. 4 resulting products


SearchResult([EOProduct(id=S2A_MSIL1C_20200513T092031_N0209_R093_T35UNP_20200513T112445, provider=uvt),
              EOProduct(id=S2A_MSIL1C_20200513T092031_N0209_R093_T35UMP_20200513T112445, provider=uvt),
              EOProduct(id=S2A_MSIL1C_20200513T092031_N0209_R093_T35TNN_20200513T112445, provider=uvt),
              EOProduct(id=S2A_MSIL1C_20200513T092031_N0209_R093_T35TMN_20200513T112445, provider=uvt)])

Available assets from the first filtered product

In [7]:
prods_S2L1C_filtered[0].assets

{'INSPIRE.xml': {'href': 'https://eodata.sage.uvt.ro/Sentinel/Sentinel-2/2020/5/13/T35/S2A_MSIL1C_20200513T092031_N0209_R093_T35UNP_20200513T112445.SAFE/INSPIRE.xml', 'type': 'application/xml', 'roles': ['metadata'], 'title': 'INSPIRE.xml'}, 'MTD_DS.xml': {'href': 'https://eodata.sage.uvt.ro/Sentinel/Sentinel-2/2020/5/13/T35/S2A_MSIL1C_20200513T092031_N0209_R093_T35UNP_20200513T112445.SAFE/DATASTRIP/DS_SGS__20200513T112445_S20200513T092634/MTD_DS.xml', 'type': 'application/xml', 'roles': ['metadata'], 'title': 'MTD_DS.xml'}, 'MTD_MSIL1C.xml': {'href': 'https://eodata.sage.uvt.ro/Sentinel/Sentinel-2/2020/5/13/T35/S2A_MSIL1C_20200513T092031_N0209_R093_T35UNP_20200513T112445.SAFE/MTD_MSIL1C.xml', 'type': 'application/xml', 'roles': ['metadata'], 'title': 'MTD_MSIL1C.xml'}, 'MTD_TL.xml': {'href': 'https://eodata.sage.uvt.ro/Sentinel/Sentinel-2/2020/5/13/T35/S2A_MSIL1C_20200513T092031_N0209_R093_T35UNP_20200513T112445.SAFE/GRANULE/L1C_T35UNP_A025544_20200513T092634/MTD_TL.xml', 'type': 'application/xml', 'roles': ['metadata'], 'title': 'MTD_TL.xml'}, 'T35UNP_20200513T092031_B01.tif': {'href': 'https://eodata.sage.uvt.ro/Sentinel/Sentinel-2/2020/5/13/T35/S2A_MSIL1C_20200513T092031_N0209_R093_T35UNP_20200513T112445.SAFE/GRANULE/L1C_T35UNP_A025544_20200513T092634/IMG_DATA/T35UNP_20200513T092031_B01.tif', 'type': 'image/tiff; application=geotiff; profile=cloud-optimized', 'roles': ['data'], 'title': 'T35UNP_20200513T092031_B01.tif', 'eo:bands': [{'name': 'B01', 'common_name': 'coastal', 'description': 'Band 1 - Coastal aerosol', 'center_wavelength': 0.443, 'full_width_half_max': 0.027}], 'proj:bbox': [499980.0, 5290200.0, 609780.0, 5400000.0], 'proj:epsg': 32635, 'proj:shape': [1830, 1830], 'proj:transform': [60.0, 0.0, 499980.0, 0.0, -60.0, 5400000.0, 0.0, 0.0, 1.0]}, 'T35UNP_20200513T092031_B02.tif': {'href': 'https://eodata.sage.uvt.ro/Sentinel/Sentinel-2/2020/5/13/T35/S2A_MSIL1C_20200513T092031_N0209_R093_T35UNP_20200513T112445.SAFE/GRANULE/L1C_T35UNP_A025544_20200513T092634/IMG_DATA/T35UNP_20200513T092031_B02.tif', 'type': 'image/tiff; application=geotiff; profile=cloud-optimized', 'roles': ['data'], 'title': 'T35UNP_20200513T092031_B02.tif', 'eo:bands': [{'name': 'B02', 'common_name': 'blue', 'description': 'Band 2 - Blue', 'center_wavelength': 0.49, 'full_width_half_max': 0.098}], 'proj:bbox': [499980.0, 5290200.0, 609780.0, 5400000.0], 'proj:epsg': 32635, 'proj:shape': [10980, 10980], 'proj:transform': [10.0, 0.0, 499980.0, 0.0, -10.0, 5400000.0, 0.0, 0.0, 1.0]}, 'T35UNP_20200513T092031_B03.tif': {'href': 'https://eodata.sage.uvt.ro/Sentinel/Sentinel-2/2020/5/13/T35/S2A_MSIL1C_20200513T092031_N0209_R093_T35UNP_20200513T112445.SAFE/GRANULE/L1C_T35UNP_A025544_20200513T092634/IMG_DATA/T35UNP_20200513T092031_B03.tif', 'type': 'image/tiff; application=geotiff; profile=cloud-optimized', 'roles': ['data'], 'title': 'T35UNP_20200513T092031_B03.tif', 'eo:bands': [{'name': 'B03', 'common_name': 'green', 'description': 'Band 3 - Green', 'center_wavelength': 0.56, 'full_width_half_max': 0.045}], 'proj:bbox': [499980.0, 5290200.0, 609780.0, 5400000.0], 'proj:epsg': 32635, 'proj:shape': [10980, 10980], 'proj:transform': [10.0, 0.0, 499980.0, 0.0, -10.0, 5400000.0, 0.0, 0.0, 1.0]}, 'T35UNP_20200513T092031_B04.tif': {'href': 'https://eodata.sage.uvt.ro/Sentinel/Sentinel-2/2020/5/13/T35/S2A_MSIL1C_20200513T092031_N0209_R093_T35UNP_20200513T112445.SAFE/GRANULE/L1C_T35UNP_A025544_20200513T092634/IMG_DATA/T35UNP_20200513T092031_B04.tif', 'type': 'image/tiff; application=geotiff; profile=cloud-optimized', 'roles': ['data'], 'title': 'T35UNP_20200513T092031_B04.tif', 'eo:bands': [{'name': 'B04', 'common_name': 'red', 'description': 'Band 4 - Red', 'center_wavelength': 0.665, 'full_width_half_max': 0.038}], 'proj:bbox': [499980.0, 5290200.0, 609780.0, 5400000.0], 'proj:epsg': 32635, 'proj:shape': [10980, 10980], 'proj:transform': [10.0, 0.0, 499980.0, 0.0, -10.0, 5400000.0, 0.0, 0.0, 1.0]}, 'T35UNP_20200513T092031_B05.tif': {'href': 'https://eodata.s

### Advanced search using query or filter extensions

Some STAC-compliant APIs support advanced search criteria using STAC [query](https://github.com/stac-api-extensions/query) extension
or STAC [filter](https://github.com/stac-api-extensions/filter) extension.

Using [custom query parameters](../api_user_guide/3_search.ipynb#Custom-query-parameters) allows you to pass raw `query` or `filter` objects directly to the backend, without any transformation by EODAG.

<div class="alert alert-info">

Note

Only `query` and `filter` for STAC providers are sent as is to the provider. Other custom parameters would be sent in a `query` object to the STAC provider.
</div>

Example: using the query extension to filter on snow/ice percentage > 99%

In [8]:
products_query = dag.search(
    provider="uvt",
    collection="sentinel-2-l1c",
    query={"s2:snow_ice_percentage": {"gt": 99}},
)
products_query

2025-10-19 16:49:52,529 eodag.core                       [INFO    ] Searching on provider uvt
2025-10-19 16:49:52,533 eodag.search.qssearch            [INFO    ] Fetching queryables: https://stac.sage.uvt.ro/search/../collections/sentinel-2-l1c/queryables
2025-10-19 16:49:53,047 eodag.search.qssearch            [INFO    ] Sending search request: https://stac.sage.uvt.ro/search


SearchResult([EOProduct(id=S2B_MSIL1C_20250225T085839_N0511_R007_T35TMK_20250225T105420, provider=uvt),
              EOProduct(id=S2B_MSIL1C_20240122T090159_N0510_R007_T35UNP_20240122T095509, provider=uvt),
              EOProduct(id=S2A_MSIL1C_20240117T090311_N0510_R007_T35UNP_20240117T094958, provider=uvt),
              EOProduct(id=S2B_MSIL1C_20230130T091129_N0509_R050_T35TPJ_20230130T094735, provider=uvt)])

## STAC Static catalog client

EODAG can search for items over a STAC static catalog using [StaticStacSearch](../../plugins_reference/generated/eodag.plugins.search.static_stac_search.StaticStacSearch.rst) plugin. Path to the catalog (or item) must be set as `search.api_endpoint` and the download plugin can also be set ([HTTPDownload](../../plugins_reference/generated/eodag.plugins.download.http.HTTPDownload.rst) by default) depending on the provider.

Here is an example with a catalog from https://stacindex.org/catalogs/spot-orthoimages-canada-2005, which will use 
 `HTTPDownload` as download plugin, without credentials as no authentication is needed for download.
 
See [download plugins documentation](../../plugins_reference/download.rst) for other available plugins.

<div class="alert alert-warning">

Warning

Please note that `StaticStacSearch` plugin development is still at an early stage. If search is too slow using this plugin, please use a catalog with less elements.

</div>

In [9]:
# Decrease logging level
setup_logging(verbose=1)

# add the provider
dag.add_provider(
    "stac_http_provider", 
    search={
        "type": "StaticStacSearch", 
        "api_endpoint": "https://canada-spot-ortho.s3.amazonaws.com/canada_spot_orthoimages/canada_spot5_orthoimages/S5_2007/catalog.json",
        "timeout": 60,
    },
)

Let's perform search :

In [12]:
from shapely.geometry import Polygon

search_polygon = Polygon([(-70, 45), (-75, 47), (-80, 47), (-80, 44)])
query_args = {
    "provider": "stac_http_provider",
    "collection": "canada_spot5_orthoimages",
    "start": "2007-05-01", 
    "end": "2007-05-06" , 
    "geom": search_polygon,
}
products = dag.search(**query_args)
products

SearchResult([EOProduct(id=S5_07351_4507_20070918, provider=stac_http_provider),
              EOProduct(id=S5_07629_4536_20070923, provider=stac_http_provider),
              EOProduct(id=S5_07129_4536_20070924, provider=stac_http_provider),
              EOProduct(id=S5_07619_4605_20070923, provider=stac_http_provider),
              EOProduct(id=S5_07702_4605_20070505, provider=stac_http_provider),
              EOProduct(id=S5_07713_4536_20070505, provider=stac_http_provider),
              EOProduct(id=S5_07050_4536_20070829, provider=stac_http_provider),
              EOProduct(id=S5_07525_4438_20070828, provider=stac_http_provider),
              EOProduct(id=S5_07600_4507_20070902, provider=stac_http_provider),
              EOProduct(id=S5_07340_4536_20070918, provider=stac_http_provider),
              EOProduct(id=S5_07904_4438_20070811, provider=stac_http_provider),
              EOProduct(id=S5_07060_4507_20070829, provider=stac_http_provider),
              EOProduct(id=S5_07916_4409_20070811, provider=stac_http_provider),
              EOProduct(id=S5_07957_4409_20070906, provider=stac_http_provider),
              EOProduct(id=S5_07703_4409_20070907, provider=stac_http_provider),
              EOProduct(id=S5_07724_4507_20070505, provider=stac_http_provider),
              EOProduct(id=S5_07724_4634_20070827, provider=stac_http_provider),
              EOProduct(id=S5_07314_4634_20070807, provider=stac_http_provider),
              EOProduct(id=S5_07944_4634_20071118, provider=stac_http_provider),
              EOProduct(id=S5_07609_4634_20070923, provider=stac_http_provider),
              EOProduct(id=S5_07622_4409_20070626, provider=stac_http_provider),
              EOProduct(id=S5_07245_4605_20070521, provider=stac_http_provider),
              EOProduct(id=S5_07711_4702_20070827, provider=stac_http_provider),
              EOProduct(id=S5_07139_4507_20070924, provider=stac_http_provider),
              EOProduct(id=S5_07900_4634_20070801, provider=stac_http_provider),
              EOProduct(id=S5_07610_4438_20070626, provider=stac_http_provider),
              EOProduct(id=S5_07515_4507_20070828, provider=stac_http_provider)])

Plot products and search polygon on map

In [13]:
import ipyleaflet as ipyl

m = ipyl.Map(center=(45, -75), zoom=5)

polygon_layer = ipyl.GeoJSON(data=search_polygon.__geo_interface__, style=dict(color='blue'))
m.add_layer(polygon_layer)

items_layer = ipyl.GeoJSON(data=products.as_geojson_object(), style=dict(color='green'))
m.add_layer(items_layer)
m

Map(center=[45, -75], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_te…

Check last product assets

In [14]:
products[-1].assets

{'m20_1_lcc00_cog.tif': {'href': 'https://canada-spot-ortho.s3.amazonaws.com/canada_spot_orthoimages/canada_spot5_orthoimages/S5_2007/S5_07515_4507_20070828/S5_07515_4507_20070828_m20_1_lcc00_cog.tif', 'type': 'image/tiff; application=geotiff; profile=cloud-optimized', 'title': 'm20_1_lcc00_cog.tif', 'roles': ['data']}, 'm20_2_lcc00_cog.tif': {'href': 'https://canada-spot-ortho.s3.amazonaws.com/canada_spot_orthoimages/canada_spot5_orthoimages/S5_2007/S5_07515_4507_20070828/S5_07515_4507_20070828_m20_2_lcc00_cog.tif', 'type': 'image/tiff; application=geotiff; profile=cloud-optimized', 'title': 'm20_2_lcc00_cog.tif', 'roles': ['data']}, 'm20_3_lcc00_cog.tif': {'href': 'https://canada-spot-ortho.s3.amazonaws.com/canada_spot_orthoimages/canada_spot5_orthoimages/S5_2007/S5_07515_4507_20070828/S5_07515_4507_20070828_m20_3_lcc00_cog.tif', 'type': 'image/tiff; application=geotiff; profile=cloud-optimized', 'title': 'm20_3_lcc00_cog.tif', 'roles': ['data']}, 'm20_4_lcc00_cog.tif': {'href': 'https://canada-spot-ortho.s3.amazonaws.com/canada_spot_orthoimages/canada_spot5_orthoimages/S5_2007/S5_07515_4507_20070828/S5_07515_4507_20070828_m20_4_lcc00_cog.tif', 'type': 'image/tiff; application=geotiff; profile=cloud-optimized', 'title': 'm20_4_lcc00_cog.tif', 'roles': ['data']}, 'p10_1_lcc00_cog.tif': {'href': 'https://canada-spot-ortho.s3.amazonaws.com/canada_spot_orthoimages/canada_spot5_orthoimages/S5_2007/S5_07515_4507_20070828/S5_07515_4507_20070828_p10_1_lcc00_cog.tif', 'type': 'image/tiff; application=geotiff; profile=cloud-optimized', 'title': 'p10_1_lcc00_cog.tif', 'roles': ['data']}, 's5_07515_4507_20070828_tn.jpg': {'href': 'https://canada-spot-ortho.s3.amazonaws.com/canada_spot_orthoimages/canada_spot5_orthoimages/S5_2007/S5_07515_4507_20070828/s5_07515_4507_20070828_tn.jpg', 'type': 'image/jpeg', 'title': 's5_07515_4507_20070828_tn.jpg', 'roles': ['auxiliary']}}

Download first asset:

In [15]:
first_asset = next(iter(products[-1].assets.values()))
first_asset

{'href': 'https://canada-spot-ortho.s3.amazonaws.com/canada_spot_orthoimages/canada_spot5_orthoimages/S5_2007/S5_07515_4507_20070828/S5_07515_4507_20070828_m20_1_lcc00_cog.tif', 'type': 'image/tiff; application=geotiff; profile=cloud-optimized', 'title': 'm20_1_lcc00_cog.tif', 'roles': ['data']}

In [16]:
path = first_asset.download(output_dir="/tmp")
path

0.00B [00:00, ?B/s]

'/tmp/S5_07515_4507_20070828'